In [1]:
%matplotlib inline

# PACKAGE IMPORT
import sys
import json
import numpy as np
import scipy as sp
import math
import xlrd
import csv
import pandas as pd
from pandas import DataFrame as df
import sklearn
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.path import Path
from mpl_toolkits.mplot3d import Axes3D
from astropy import units as u
from astropy.coordinates import Angle, SkyCoord
from astropy.table import Table, vstack

import seaborn as sns
sns.set()
sns.set_style('darkgrid')
sns.set_context('poster')

print('PACKAGES IMPORTED')

PACKAGES IMPORTED


In [2]:
#Import FRB data
data_FRB_raw = pd.read_csv('frbcat_20190722.csv')

name_FRB = data_FRB_raw['frb_name']
RA_FRB = data_FRB_raw['rop_gl']
DEC_FRB = data_FRB_raw['rop_gb']
DM_FRB = data_FRB_raw['rmp_dm'].str.split('&').str[0]

df_FRB = pd.DataFrame({'FRB Name': name_FRB, 'FRB RA': RA_FRB, 'FRB DEC': DEC_FRB, 'FRB DM': DM_FRB})

df_FRB.head()

,FRB Name,FRB RA,FRB DEC,FRB DM
0,FRB190523,117.030000,44.000000,760.8
1,FRB181228,253.391500,-26.063300,354.2
2,FRB181017,50.500000,-47.000000,239.97
3,FRB181016,345.510000,22.670000,1982.8
4,FRB180924,0.742467,-49.414787,361.42


In [3]:
with open('DM_cat_v1.56.dat','r') as f:
    next(f) # skip first row
    df_pulsar_raw = pd.DataFrame(l.rstrip().split() for l in f)

new_header_pulsar_raw = df_pulsar_raw.iloc[0] #grab the first row for the header
df_pulsar_raw = df_pulsar_raw[1:] #take the data less the header row
df_pulsar_raw.columns = new_header_pulsar_raw

name_pulsar = df_pulsar_raw['PSRJ']
RA_pulsar_hms = df_pulsar_raw['RAJ'] #denotes hr:min:sec
DEC_pulsar_hms = df_pulsar_raw['DECJ']
DM_pulsar_hms = df_pulsar_raw['DM']

df_pulsar_hms = pd.DataFrame({'Pulsar Name': name_pulsar, 'Pulsar RA': RA_pulsar_hms, 'Pulsar DEC': DEC_pulsar_hms, 'Pulsar DM': DM_pulsar_hms})
df_pulsar_hms = df_pulsar_hms[df_pulsar_hms.notnull()].dropna()

#convert to galactic coords
def to_degrees(df_name):
    return Angle(df_name, unit=u.deg).degree

DEC_pulsar = to_degrees(df_pulsar_hms['Pulsar DEC'])
RA_pulsar = to_degrees(df_pulsar_hms['Pulsar RA'])

df_pulsar = pd.DataFrame({'Pulsar Name' : df_pulsar_hms['Pulsar Name'], 'Pulsar RA': RA_pulsar, 'Pulsar DEC': DEC_pulsar, 'Pulsar DM': df_pulsar_hms['Pulsar DM']}).reset_index(drop=True)

df_pulsar.head()

,Pulsar Name,Pulsar RA,Pulsar DEC,Pulsar DM
0,J0006+1834,0.101333,18.583056,11.41
1,J0011+08,0.192778,8.166667,24.9
2,J0014+4746,0.238264,47.775944,30.405
3,J0023+0923,0.388022,9.389964,14.3283
4,J0024-7204aa,0.401575,-72.081283,24.971


In [4]:
with open('rratalog.rtf','r') as f:
    df_rrat_raw = pd.DataFrame(l.rstrip().split() for l in f)

df_rrat_raw = df_rrat_raw.drop(df_rrat_raw.index[:9])

new_header_rrat_raw = df_rrat_raw.iloc[0] #grab the first row for the header
df_rrat_raw = df_rrat_raw[1:] #take the data less the header row
df_rrat_raw.columns = new_header_rrat_raw

df_rrat = pd.DataFrame({'RRat Name' : df_rrat_raw['Name'], 'RRat RA' : df_rrat_raw['l'], 'RRat DEC' : df_rrat_raw['b'], 'RRat DM' : df_rrat_raw['DM']}).reset_index(drop=True)
df_rrat = df_rrat[df_rrat.notnull()].dropna()
df_rrat.head()

<class 'str'>


,RRat Name,RRat RA,RRat DEC,RRat DM
0,J0054+69,123.20,6.56,90.3
1,J0054+66,123.18,3.96,15
2,J0156+04**,152,-55,27.5
3,J0103+54,124.74,-8.80,55.6
4,J0201+7005,128.88,8.03,21


In [5]:
#to astropy table
table_FRB = Table(df_FRB.values, names=('FRB Name', 'FRB RA', 'FRB DEC', 'FRB DM'))
table_pulsar = Table(df_pulsar.values, names=('Pulsar Name', 'Pulsar RA', 'Pulsar DEC', 'Pulsar DM'))
table_rrat = Table(df_rrat.values, names=('RRat Name', 'RRat RA', 'RRat DEC', 'RRat DM'))

alltab = vstack( (table_FRB, table_pulsar, table_rrat) )

alltab

FRB Name,FRB RA,FRB DEC,FRB DM,Pulsar Name,Pulsar RA,Pulsar DEC,Pulsar DM,RRat Name,RRat RA,RRat DEC,RRat DM
object,object,object,object,object,object,object,object,object,object,object,object
FRB190523,117.03,44.0,760.8,--,--,--,--,--,--,--,--
FRB181228,253.3915,-26.0633,354.2,--,--,--,--,--,--,--,--
FRB181017,50.5,-47.0,239.97,--,--,--,--,--,--,--,--
FRB181016,345.51,22.67,1982.8,--,--,--,--,--,--,--,--
FRB180924,0.742467,-49.414787,361.42,--,--,--,--,--,--,--,--
FRB180817.J1533+42,68.0,54.0,1006.84,--,--,--,--,--,--,--,--
FRB180814.J1554+74,108.0,37.0,238.32,--,--,--,--,--,--,--,--
FRB180814.J0422+73,136.0,16.0,189.38,--,--,--,--,--,--,--,--
FRB180812.J0112+80,123.0,18.0,802.57,--,--,--,--,--,--,--,--


In [6]:
#Make SkyCoords
FRB_skycoords = np.dstack((df_FRB['FRB RA'].astype(float),df_FRB['FRB DEC'].astype(float)))
pulsar_skycoords = np.dstack((df_pulsar['Pulsar RA'].astype(float),df_pulsar['Pulsar DEC'].astype(float)))

# print(list(pulsar_skycoords[0]))

In [15]:
FRB_RA = df_FRB['FRB RA'].astype(float).values
FRB_DEC = df_FRB['FRB DEC'].astype(float).values
FRB_name = df_FRB['FRB Name']

pulsar_RA = df_pulsar['Pulsar RA'].astype(float).values
pulsar_DEC = df_pulsar['Pulsar DEC'].astype(float).values
pulsar_name = df_pulsar['Pulsar Name']

RRat_RA = df_rrat['RRat RA'].astype(float).values
RRat_DEC = df_rrat['RRat DEC'].astype(float).values
RRat_name = df_rrat['RRat Name']

FRB_skycoords = SkyCoord(FRB_RA, FRB_DEC, frame="icrs", unit="deg")
pulsar_skycoords = SkyCoord(pulsar_RA, pulsar_DEC, frame="icrs", unit="deg")
RRat_skycoords = SkyCoord(RRat_RA, RRat_DEC, frame="icrs", unit="deg")

##match_to_cat[0] is indices into catalogcoord to get the matched points for each of this object’s coordinates. 
##match_to_cat[1] is angle of separation. 
##match_to_cat[2] 3D distance between the closest match for each element in this object in catalogcoord (all at same z axis) 

closest_matches_FRB_pulsar = FRB_skycoords.match_to_catalog_sky(pulsar_skycoords)
closest_matches_FRB_RRat = FRB_skycoords.match_to_catalog_sky(RRat_skycoords)

ang_sep_FRB_pulsar = closest_matches_FRB_pulsar[1].value
ang_sep_FRB_RRat = closest_matches_FRB_RRat[1].value

def get_matches(catalogue_skycoords, event_names, degrees):
    "Get matches between FRBs and other catalogue of interest within specified degree"
    FRB_matched = []
    event_matched = []
    FRB_coords_ra = []
    FRB_coords_dec = []
    event_coords_ra = []
    event_coords_dec = []
    closest_coords = FRB_skycoords.match_to_catalog_sky(catalogue_skycoords)
    ang_sep = closest_coords[1].value
    for i in range(len(ang_sep)):
        if ang_sep[i] < degrees:
            FRB_index = i
            FRB_matched_ = FRB_name[FRB_index]
            FRB_matched = np.append(FRB_matched,FRB_matched_)
            
            FRB_coords_ra_ = FRB_skycoords[FRB_index].ra.value
            FRB_coords_ra = np.append(FRB_coords_ra,FRB_coords_ra_)
            FRB_coords_dec_ = FRB_skycoords[FRB_index].dec.value
            FRB_coords_dec = np.append(FRB_coords_dec,FRB_coords_dec_)
            
            event_index = closest_coords[0][i]
            event_matched_ = event_names[event_index]
            event_matched = np.append(event_matched,event_matched_)
            
            event_coords_ra_ = catalogue_skycoords[event_index].ra.value
            event_coords_ra = np.append(event_coords_ra,event_coords_ra_)
            event_coords_dec_ = catalogue_skycoords[event_index].dec.value
            event_coords_dec = np.append(event_coords_dec,event_coords_dec_)
    return FRB_matched, list(zip(FRB_coords_ra,FRB_coords_dec)), event_matched, list(zip(event_coords_ra, event_coords_dec))
            
#Pulsar matches
print(get_matches(pulsar_skycoords, pulsar_name, 1)[0])
print(get_matches(pulsar_skycoords, pulsar_name, 1)[1])
print(get_matches(pulsar_skycoords, pulsar_name, 1)[2])
print(get_matches(pulsar_skycoords, pulsar_name, 1)[3])

#RRat matches
print(get_matches(RRat_skycoords, RRat_name, 2)[0])
print(get_matches(RRat_skycoords, RRat_name, 2)[1])
print(get_matches(RRat_skycoords, RRat_name, 2)[2])
print(get_matches(RRat_skycoords, RRat_name, 2)[3])


print(float(df_FRB['FRB DM'].iloc[16]),float(df_pulsar['Pulsar DM'].iloc[2506]),float(df_FRB['FRB DM'].iloc[16])-float(df_pulsar['Pulsar DM'].iloc[2506]))

['FRB180727.J1311+26' 'FRB180309' 'FRB180110' 'FRB160102']
[(25.0, 85.0), (10.9, -45.4), (7.8, -51.9), (18.9, -60.8)]
['J2353+85' 'J1045-4509' 'J0907-5157' 'J1833-6023']
[(23.9, 85.56666666666666), (10.763940822222223, -45.16503397222222), (9.121083333333333, -51.966458333333335), (18.554111111111112, -60.38444444444444)]
['FRB180714' 'FRB130626']
[(14.8, 8.72), (7.45003, 27.4203)]
['J1753-12' 'J1623-08']
[(15.48, 7.17), (5.77, 27.37)]
642.07 38.0 604.07
